# 퀀텀 AI 대회 '칸쵸' 팀 - 모델 재현 노트북

본 노트북은 `best_model.pth` 모델 가중치 파일을 로드하여 최고 점수를 재현하는 데 사용됩니다.

**실행 전 확인:**
1. `pip install -r requirements.txt`로 모든 의존성을 설치했는지 확인하세요.
2. `best_model.pth` 파일이 이 노트북과 동일한 디렉토리에 있는지 확인하세요.

## 1. 의존성 및 설정

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

import numpy as np
import pandas as pd
import pennylane as qml

import os
import random
from sklearn.metrics import accuracy_score

# 경로 설정
base_path = os.path.join(os.getcwd(), 'output')
os.makedirs(base_path, exist_ok=True)

# 난수 고정
SEED = 3006
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. 양자 회로 정의

In [ ]:
# 큐비트 기반 양자 회로 정의
dev = qml.device("default.qubit", wires=6)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    num_qubits = 6
    layers = 3
    qml.AngleEmbedding(inputs, wires=range(num_qubits))
    for l in range(layers):
        for i in range(num_qubits):
            qml.RX(weights[(l * num_qubits + i) % weights.shape[0]], wires=i)
        for i in range(0, num_qubits, 2):
            if i + 1 < num_qubits:
                qml.CNOT(wires=[i, i+1])
    for i in range(num_qubits):
        qml.RZ(weights[(i + weights.shape[0] // 2) % weights.shape[0]], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

## 3. 데이터 준비

In [ ]:
# 데이터 변환 정의
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 데이터셋 로드
test_ds = datasets.FashionMNIST(root=base_path, train=False, download=True, transform=transform_test)

# 0과 6만 필터링
test_mask  = (test_ds.targets  == 0) | (test_ds.targets  == 6)
test_idx   = torch.where(test_mask)[0]

# 레이블 이진화
test_ds.targets[ test_ds.targets  == 6] = 1

# 0과 6만 포함된 데이터셋
binary_test_ds  = Subset(test_ds,  test_idx)

## 4. 모델 정의

In [ ]:
# CNN + 양자 회로 + MLP 통합 모델
class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=4, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(64, 6)
        self.norm = nn.LayerNorm(6)
        self.q_params = nn.Parameter(torch.rand(30))
        self.fc2 = nn.Linear(6, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = self.dropout(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.norm(x)
        q_out = quantum_circuit(x, self.q_params)
        q_out = torch.stack(list(q_out), dim=1).to(torch.float32)
        x = self.fc2(q_out)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

## 5. 모델 스펙 검사

In [ ]:
# QNN 구조 및 파라미터 사양 확인
temp_model = HybridModel()
temp_model.eval()

input_sample = torch.randn(1, 6)
q_weight_sample = temp_model.q_params.data
qc_info = qml.specs(quantum_circuit)(input_sample, q_weight_sample)

# 스펙 제한 조건 확인
if qc_info["num_tape_wires"] > 8:
    raise ValueError("❌ 큐비트 수 초과")
if qc_info["resources"].depth > 30:
    raise ValueError("❌ 회로 깊이 초과")
if qc_info["num_trainable_params"] > 60:
    raise ValueError("❌ 학습 가능한 파라미터 수 초과")
print("✅ QNN 회로 스펙 체크 완료")

# 전체 학습 파라미터 수 체크
trainable_count = sum(param.numel() for param in temp_model.parameters() if param.requires_grad)
if trainable_count > 50000:
    raise ValueError(f"❌ 파라미터 수 초과: {trainable_count}")
print(f"✅ 전체 파라미터 수 검증 통과: {trainable_count}")
del temp_model

## 6. 모델 로드 및 정확도 재현

In [ ]:
# 로컬 경로에서 모델 파라미터 로드
model_path = "best_model_seed_3006.pth"

if not os.path.exists(model_path):
    print(f"❗️ {model_path} 파일이 없습니다. 리포지토리에서 파일을 다운로드 받아주세요.")
else:
    best_model = HybridModel().to(device)
    # map_location을 통해 현재 설정된 device로 모델을 로드합니다.
    best_model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"✅ 모델 파라미터 업로드 완료! ({model_path})")

    # 테스트 데이터 로더 생성
    test_loader  = DataLoader(binary_test_ds,  batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

    # 평가 시작
    best_model.eval()
    test_correct, test_total = 0, 0
    all_preds = []
    with torch.no_grad():
        for imgs, lbls in test_loader:
            # 원본 코드의 오류 수정: lbls도 device로 이동해야 합니다.
            imgs, lbls = imgs.to(device), lbls.to(device)
            
            out = best_model(imgs)
            pred = out.argmax(dim=1)
            test_correct += (pred == lbls).sum().item()
            test_total += lbls.size(0)
            all_preds.extend(pred.cpu().numpy())

    test_acc = test_correct / test_total * 100
    print("="*40)
    print(f"🎯 최종 재현 정확도 (0/6) = {test_acc:.4f}%")
    print("="*40)

    # (선택 사항) 제출 파일 생성 로직
    best_preds = [0 if p == 0 else 6 for p in all_preds]
    y_pred_full = np.zeros(len(test_ds), dtype=int)
    y_pred_full[test_idx] = best_preds
    print(f"전체 테스트셋 대상 예측 생성 완료 (길이: {len(y_pred_full)})")

    # df = pd.DataFrame({"y_pred": y_pred_full})
    # csv_name = os.path.join(base_path, f"y_pred_{SEED}_reproduce.csv")
    # df.to_csv(csv_name, index=False, header=False)
    # print(f"제출 파일 저장 완료: {csv_name}"

---

## [부록] 모델 학습 코드 (참고용)

아래는 `best_model.pth` 파일을 생성하는 데 사용된 학습 코드입니다. (원본 노트북 기준 주석 처리된 부분)

### A. 학습용 데이터 준비 (원본)

In [ ]:
# # 데이터 변환 정의
# transform_train = transforms.Compose([
#     transforms.RandomRotation(15),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
# ])
# transform_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
# ])

# # 데이터셋 로드
# train_ds = datasets.FashionMNIST(root=base_path, train=True, download=True, transform=transform_train)
# test_ds = datasets.FashionMNIST(root=base_path, train=False, download=True, transform=transform_test)

# # 0과 6만 필터링
# train_mask = (train_ds.targets == 0) | (train_ds.targets == 6)
# test_mask  = (test_ds.targets  == 0) | (test_ds.targets  == 6)
# train_idx  = torch.where(train_mask)[0]
# test_idx   = torch.where(test_mask)[0]

# # 레이블 이진화
# train_ds.targets[train_ds.targets == 6] = 1
# test_ds.targets[ test_ds.targets  == 6] = 1

# # 0과 6만 포함된 데이터셋
# binary_train_ds = Subset(train_ds, train_idx)
# binary_test_ds  = Subset(test_ds,  test_idx)

### B. 학습 루프 (원본)

In [ ]:
# # 데이터 로더 설정
# train_loader = DataLoader(binary_train_ds, batch_size=64, shuffle=True,  num_workers=0, pin_memory=True)
# test_loader  = DataLoader(binary_test_ds,  batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

# # 모델·옵티마이저·스케줄러·손실함수 설정
# model = HybridModel().to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.2)
# criterion = nn.NLLLoss()

# best_acc, epochs_no_improve = 0.0, 0
# early_stopping_patience = 70
# best_model_path   = os.path.join(base_path, f'best_model_seed_{SEED}.pth')

# for epoch in range(800):
#     model.train()
#     total_loss, correct, total = 0, 0, 0
#     for imgs, lbls in train_loader:
#         imgs, lbls = imgs.to(device), lbls.to(device)
#         optimizer.zero_grad()
#         out = model(imgs)
#         loss = criterion(out, lbls)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#         preds = out.argmax(dim=1)
#         correct += (preds == lbls).sum().item()
#         total += lbls.size(0)
#     train_acc = correct / total
#     scheduler.step(train_acc)
#     if train_acc > best_acc:
#         best_acc = train_acc
#         epochs_no_improve = 0
#         torch.save(model.state_dict(), best_model_path)
#     else:
#         epochs_no_improve += 1
#         if epochs_no_improve >= early_stopping_patience:
#             break
